In [1]:
import pandas as pd
import numpy as np
import textatistic as tx
import os
import re
import spacy
import sklearn

from threading import Thread
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from spacy.lang.en.stop_words import STOP_WORDS


In [2]:
#   DATA CLEANING

neg_data = []
pos_data = []

dir_neg = r"C:\Users\KingED\Documents\Random\NLP_init\test\neg"
dir_pos = r"C:\Users\KingED\Documents\Random\NLP_init\test\pos"

def maker(loc: list, path: str):
    for files in os.listdir(path):
        os.chdir(path)
        with open(files, 'rb+') as file:
            loc.append(file.readlines())
       
maker(neg_data, dir_neg)
maker(pos_data, dir_pos)


pattern_1 = r"^b\"|^b\'"
pattern_2 = r"<br\s\/\>"
patter_3 = r'\\|\"|\''

pos_word = []
neg_word = []

#FIXME : convert these loops into functions
for c in pos_data:
    word = str(c[0])
    word = re.sub(pattern_2, "", word)
    word = re.sub(pattern_1, "", word)
    word = re.sub(patter_3, "", word)
    pos_word.append(word)
    
for c in neg_data:
    word = str(c[0])
    word = re.sub(pattern_2, "", word)
    word = re.sub(pattern_1, "", word)
    word = re.sub(patter_3, "", word)
    neg_word.append(word)

 

In [3]:
# FIXME: make this function compute the gunningfog and flesch scores at the same time 

fog_scores = []
flesch_score = []
issues = []
for x in pos_word:
    try:
        fog = tx.gunningfog_score(x)
        flesch = tx.flesch_score(x)
        fog_scores.append(fog)
        flesch_score.append(flesch)
      
    except ZeroDivisionError:
        issues.append(x) 


In [4]:
average_fog = sum(fog_scores)/ len(fog_scores)
average_flesch = sum(flesch_score)/ len(flesch_score)

print(f"the average flesch score for poistive reviews is {average_flesch}")
print(f"the average runningfog scores for the postive reviews is {average_fog}")

the average flesch score for poistive reviews is 78.17089281827614
the average runningfog scores for the postive reviews is 9.868059804420177


In [5]:
doc_pos = pd.DataFrame({'review': pos_word, 'score': 1})
doc_neg = pd.DataFrame({'review': neg_word, 'score': 0})

data  = pd.concat([doc_neg, doc_pos], ignore_index=True)
data

,review,score
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0
...,...,...
24995,I was extraordinarily impressed by this film. ...,1
24996,"Although Im not a golf fan, I attended a sneak...",1
24997,"From the start of The Edge Of Love, the viewer...",1
24998,"This movie, with all its complexity and subtle...",1


In [6]:
os.chdir(r"C:\Users\KingED\Documents\Random\NLP_init")

for file in os.listdir():
    if file != 'review.csv':
        data.to_csv('review.csv')
    else:
        print("file aready exists")

file aready exists


In [7]:
data_array = data.values
data_array[0, :]

array(['Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costners character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks hes better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutchers ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.',
       0], dtype=object)

In [8]:
"""
TODO: TOKENIZATION
TODO: REMOVE STOPWORDS
TODO: LEMMATIZE
TODO: WORD AND CHARACTER COUNT

"""
def token_maker(word: list) -> list:
    words = []
    for x in range(len(pos_word)):
        words.append(word_tokenize(word[x]))
    return words

pos_tokens = token_maker(pos_word)
neg_tokens = token_maker(neg_word)



In [9]:
# SPACEY
"""
TODO: NER
TODO: POS TAGGING
"""

# nlp = spacy.load("en_core_web_sm")
# doc = nlp(doc)

# def ner_pos():
#     ners = [(token.text, token.label_) for word in doc]
#     tags = [(token.text, token.tags_) for word in doc]
    
word = word_tokenize(pos_word[1])

lemma = WordNetLemmatizer()
tfid = TfidfVectorizer()

# lemma.lemmatize(word)
tfidword = tfid.fit_transform(data['review'])

In [12]:
counter = CountVectorizer(lowercase=True,
                          strip_accents='ascii',
                          tokenizer=None,
                          stop_words=list(STOP_WORDS),
                          )

encoder = LabelEncoder()
data['score'] = encoder.fit_transform(data['score'])

x = data['review'].values
y = data['score'].values
xtrain, xtest, ytrain, ytest = train_test_split(x, y, shuffle=True, test_size=0.3, random_state=34)


new_xtrain = counter.fit_transform(xtrain)
new_xtest = counter.transform(xtest)

print(new_xtrain.shape)

model = MultinomialNB(alpha=1)
model.fit(new_xtrain, ytrain)
pred = model.predict(new_xtest)
print(f"the accuracy of the model is: {accuracy_score(pred, ytest) * 100}")

c:\Users\KingED\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 'nt', 've'] not in stop_words.
  warnings.warn(


(17500, 68662)
the accuracy of the model is: 87.14666666666666
